In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 6.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [5]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://subcritical-readily-mahalia.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://subcritical-readily-mahalia.ngrok-free.dev


True

In [6]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [7]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），簡稱明新科大，是一所位於臺灣新竹縣新豐鄉的私立科技大學。該校秉持「堅毅、求新、創造」的校訓，以「深耕在地、放眼國際」為願景，致力於培育具備實務經驗與人文素養的專業人才，並以成為「一流產業大學」為發展目標。

**歷史沿革**
明新科技大學的創辦可追溯至1966年3月1日，當時以「明新工業專科學校」立案，初期設有機械、土木、工業管理三科，首屆招收300名學生。 隨著時代演進和教育部的核准，學校經歷多次改制：
*   1993年，更名為「私立明新工商專科學校」。
*   1997年7月，改制為「明新技術學院」，並設有專科部。
*   2002年8月，奉教育部核准優先升格為「明新科技大學」。
*   2018年12月，更名為「明新學校財團法人明新科技大學」。

**學術與特色**
明新科技大學現設有六個學院，包括半導體學院、工程學院、管理學院、民生學院、人文與設計學院、共同教育學院。 涵蓋20個學系、2個學位學程（含1個博士學位學程）及11個碩士班，致力於培養「跨域整合、務實創新、全人學習」的專業人才。

該校鄰近新竹科學園區與工業區，地理位置優越，因此多年來與產業界建立了緊密的產學合作關係，為區域產業培養了大量優秀人才。 明新科大尤其在半導體和AI領域表現突出，積極建置「半導體封裝測試類產線示範工廠」及「半導體產業設備廠務與檢測人才培育基地」，並設有技職體系中第一座「半導體學院」，更獲准設立「半導體科技博士學位學程」，以培育業界所需的半導體專業人才。 此外，學校也積極發展「智慧機械」、「永續智慧商務」等特色領域，並推廣AI科技應用與英語教學，提升學生的國際移動力與全球學習視野。

**辦學績效與榮譽**
明新科技大學的畢業生深受企業肯定，在《遠見》雜誌「2022企業最愛公私立技職科大調查」中，明新科大在「工程」和「資訊」兩大領域入榜企業最愛私立科大第二名，且畢業生起薪更是私立科大之冠，甚至優於部分國立科大。 學校也以國際化為亮點，國際學生人數居全國之首，打造了多元的校園環境。 明新科大在2025年《大學辦學績效校長互評卓越獎》中排名亦有顯著提升。


In [ ]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學的現任校長是呂明峯教授。他於2025年2月1日正式上任，成為明新科技大學的第11任校長。

呂明峯校長在半導體教育和產業實務方面擁有豐富經驗，並曾打造全台首座半導體封裝測試類產線，同時也推動成立了半導體學院。 他上任後提出了「四大核心模組」的校務治理策略，旨在將明新科大打造成為「具國際魅力的產業科技大學」。


In [8]:
from flask import Flask, request, abort
import logging
import os
import time
from google.genai import types

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FileMessageContent
)

app = Flask(__name__)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
app.logger.setLevel(logging.INFO)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

# 儲存檔案的目錄
UPLOAD_DIR = "/content/uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# 儲存每個使用者的對話 session 和上傳的檔案
user_sessions = {}  # {user_id: {"chat": chat_object, "uploaded_file": gemini_file}}

def get_user_session(user_id):
    """取得或建立使用者的對話 session"""
    if user_id not in user_sessions:
        # 建立新的對話 session
        new_chat = client.chats.create(
            model="gemini-2.5-flash",
            config=GenerateContentConfig(
                system_instruction="你是一個中文的AI助手，請用繁體中文回答。如果使用者有提供參考文件，請根據文件內容回答問題。",
                tools=[google_search_tool],
                response_modalities=["TEXT"],
            )
        )
        user_sessions[user_id] = {
            "chat": new_chat,
            "uploaded_file": None
        }
    return user_sessions[user_id]

def download_line_file(message_id, file_name):
    """從 LINE 下載使用者上傳的檔案"""
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        file_content = line_bot_blob_api.get_message_content(message_id)

        file_path = os.path.join(UPLOAD_DIR, file_name)

        with open(file_path, 'wb') as f:
            f.write(file_content)

        return file_path

def upload_file_to_gemini(file_path):
    """上傳檔案到 Gemini Files API"""
    uploaded_file = client.files.upload(
        file=file_path,
        config={'display_name': os.path.basename(file_path)}
    )

    # 等待檔案處理完成
    while uploaded_file.state.name == "PROCESSING":
        print("檔案處理中...")
        time.sleep(1)
        uploaded_file = client.files.get(name=uploaded_file.name)

    if uploaded_file.state.name == "FAILED":
        raise Exception("檔案上傳處理失敗")

    return uploaded_file

def query_with_rag(user_id, question):
    """使用 RAG 模式回答問題"""
    session = get_user_session(user_id)
    uploaded_file = session["uploaded_file"]

    if uploaded_file:
        # 有上傳檔案，使用 RAG 模式
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_uri(
                            file_uri=uploaded_file.uri,
                            mime_type=uploaded_file.mime_type
                        ),
                        types.Part.from_text(text=f"請根據上述提供的檔案內容，用繁體中文回答這個問題：{question}")
                    ]
                )
            ]
        )
        return response.text
    else:
        # 沒有上傳檔案，使用一般多輪對話
        response = session["chat"].send_message(message=question)
        return response.text

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    """處理文字訊息"""
    text = event.message.text
    user_id = event.source.user_id

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        if text.startswith('AI '):
            prompt = text[3:]
            try:
                # 使用 RAG 或一般對話
                reply_text = query_with_rag(user_id, prompt)

                # 檢查是否有上傳檔案，加上提示
                session = get_user_session(user_id)
                if session["uploaded_file"]:
                    reply_text = f"📄 [RAG 模式]\n\n{reply_text}"

                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=reply_text)]
                    )
                )
            except Exception as e:
                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=f"❌ 發生錯誤：{str(e)}")]
                    )
                )
        elif text == "清除文件":
            # 清除使用者上傳的檔案
            session = get_user_session(user_id)
            session["uploaded_file"] = None
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="✅ 已清除上傳的文件，恢復一般對話模式。")]
                )
            )
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="請輸入「AI 問題」來開始對話\n或上傳 TXT/PDF 檔案啟用 RAG 模式")]
                )
            )

@handler.add(MessageEvent, message=FileMessageContent)
def handle_file_message(event):
    """處理使用者上傳的檔案"""
    user_id = event.source.user_id
    file_name = event.message.file_name

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # 檢查檔案類型
        if not (file_name.endswith('.txt') or file_name.endswith('.pdf')):
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="⚠️ 目前只支援 TXT 或 PDF 檔案")]
                )
            )
            return

        try:
            # 下載檔案
            file_path = download_line_file(event.message.id, file_name)
            print(f"檔案已下載：{file_path}")

            # 上傳到 Gemini
            uploaded_file = upload_file_to_gemini(file_path)
            print(f"檔案已上傳到 Gemini：{uploaded_file.uri}")

            # 儲存到使用者 session
            session = get_user_session(user_id)
            session["uploaded_file"] = uploaded_file

            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"✅ 檔案「{file_name}」上傳成功！\n\n現在您可以輸入「AI 問題」來詢問關於這份文件的問題。\n\n輸入「清除文件」可恢復一般對話模式。")]
                )
            )
        except Exception as e:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"❌ 檔案處理失敗：{str(e)}")]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Request body: {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"file","id":"595032708586144024","markAsReadToken":"DlOSm6AnmWNz8S5JuUQADTT9_9h4XZERa1Rk6koOmmBoCIu_mHoI4zSSSqnS3g7b5HJJJfRYJAwI-4OkHkQQTUS1tvPyCqPBtVxD-Vaovx0gLKkodp3n8nxMdYRC9XH-Y3ECJn_ygsX3J2tepyYxffLV09tvkFfuz3XOdseocrJqD9WMQAeIFdB90X9xDU_W9OoB6BRJf0OSb76Awq365A","fileName":"RAG_reference.pdf","fileSize":24399,"contentProvider":{"type":"line"}},"webhookEventId":"01KE3HVM1V5TZ1J4DTSEPA8VZK","deliveryContext":{"isRedelivery":false},"timestamp":1767498436370,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"b8b66166f68d4fdf9eab02f492b65939","mode":"active"}]}


BODY:  {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"file","id":"595032708586144024","markAsReadToken":"DlOSm6AnmWNz8S5JuUQADTT9_9h4XZERa1Rk6koOmmBoCIu_mHoI4zSSSqnS3g7b5HJJJfRYJAwI-4OkHkQQTUS1tvPyCqPBtVxD-Vaovx0gLKkodp3n8nxMdYRC9XH-Y3ECJn_ygsX3J2tepyYxffLV09tvkFfuz3XOdseocrJqD9WMQAeIFdB90X9xDU_W9OoB6BRJf0OSb76Awq365A","fileName":"RAG_reference.pdf","fileSize":24399,"contentProvider":{"type":"line"}},"webhookEventId":"01KE3HVM1V5TZ1J4DTSEPA8VZK","deliveryContext":{"isRedelivery":false},"timestamp":1767498436370,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"b8b66166f68d4fdf9eab02f492b65939","mode":"active"}]}
檔案已下載：/content/uploaded_files/RAG_reference.pdf
檔案已上傳到 Gemini：https://generativelanguage.googleapis.com/v1beta/files/adehfldgi463


INFO:werkzeug:127.0.0.1 - - [04/Jan/2026 03:47:20] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"text","id":"595032814450376894","quoteToken":"18NorOiZxpQ_tsjGuqQSdDOnS7AUqoTlTgK7YGigAvARKaLDHUSSrxi4pTsOFemDCJ4MCFkCJYnPwT0SUTTPUr7aKqmyFYd9GiJVBXQtDtlgY5boluKZM_qYh5ZzlSbF399d6vrSGgdemu0WiPdyLA","markAsReadToken":"yAFOUudLCsyLKvqyzXz_FcFI-hKXAzQq6v_riZFbvbamM5G9aJtLL9xSXXVgJhLQ_Z0QhzlP1-36uwcxSFuZIWgQnY159swPcr35SnHpJu4uBTJZpBtk5e0U-agVRaa5-0BnghNCxZaJisKSR_yyK3H_B0TsX3cootB-uPBKosXCy0pVkYaX1mVPBY-aOpUCPRZHt_cfaE_VlmHSBd7Kyg","text":"AI 我的電腦效能如何?"},"webhookEventId":"01KE3HXHVCZC71D97MW6KYK6Z3","deliveryContext":{"isRedelivery":false},"timestamp":1767498499440,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"dc47c374ca8d42bf80aab05d84b66974","mode":"active"}]}


BODY:  {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"text","id":"595032814450376894","quoteToken":"18NorOiZxpQ_tsjGuqQSdDOnS7AUqoTlTgK7YGigAvARKaLDHUSSrxi4pTsOFemDCJ4MCFkCJYnPwT0SUTTPUr7aKqmyFYd9GiJVBXQtDtlgY5boluKZM_qYh5ZzlSbF399d6vrSGgdemu0WiPdyLA","markAsReadToken":"yAFOUudLCsyLKvqyzXz_FcFI-hKXAzQq6v_riZFbvbamM5G9aJtLL9xSXXVgJhLQ_Z0QhzlP1-36uwcxSFuZIWgQnY159swPcr35SnHpJu4uBTJZpBtk5e0U-agVRaa5-0BnghNCxZaJisKSR_yyK3H_B0TsX3cootB-uPBKosXCy0pVkYaX1mVPBY-aOpUCPRZHt_cfaE_VlmHSBd7Kyg","text":"AI 我的電腦效能如何?"},"webhookEventId":"01KE3HXHVCZC71D97MW6KYK6Z3","deliveryContext":{"isRedelivery":false},"timestamp":1767498499440,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"dc47c374ca8d42bf80aab05d84b66974","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [04/Jan/2026 03:48:38] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"text","id":"595033044415938561","quoteToken":"abIW46cMidFxms89zyqGj8z44N7d_OgmBswIVGOVt391AzbSIbTwXJvQuG9L5TD1TxO7GfRBcr8HnQ0GFwHLzM8m3bavioIcuU6mIYvjKIsWWF9Ck68_JU_f1N3Nj_5zcXHU8kqkTzClDPV8fuAzMQ","markAsReadToken":"geQ2Z-PN7SxzK-p7hLPkieW2N1GHYj9QXOZnw2IzXPwdGNl0FztLZvUmAycEjHFyVZlAo76ht2HB7NssNB_b_csyp-liKYjIcUmcOJf7RNM558zd3ZWzC054oxocD9Wr-qnN25ly2zYZ-xK3gWIBXftMp0_rgkmrARhoYg9CFcyRNaA23g_TSTxO-r7-tMFWATBWgCkp9qOGzDVV7RhePg","text":"AI 我用什麼電腦螢幕"},"webhookEventId":"01KE3J1Q63E8JT6QYXBZT6SB57","deliveryContext":{"isRedelivery":false},"timestamp":1767498636424,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"88238388b6ca4bc6a8ed135cb7928c4a","mode":"active"}]}


BODY:  {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"text","id":"595033044415938561","quoteToken":"abIW46cMidFxms89zyqGj8z44N7d_OgmBswIVGOVt391AzbSIbTwXJvQuG9L5TD1TxO7GfRBcr8HnQ0GFwHLzM8m3bavioIcuU6mIYvjKIsWWF9Ck68_JU_f1N3Nj_5zcXHU8kqkTzClDPV8fuAzMQ","markAsReadToken":"geQ2Z-PN7SxzK-p7hLPkieW2N1GHYj9QXOZnw2IzXPwdGNl0FztLZvUmAycEjHFyVZlAo76ht2HB7NssNB_b_csyp-liKYjIcUmcOJf7RNM558zd3ZWzC054oxocD9Wr-qnN25ly2zYZ-xK3gWIBXftMp0_rgkmrARhoYg9CFcyRNaA23g_TSTxO-r7-tMFWATBWgCkp9qOGzDVV7RhePg","text":"AI 我用什麼電腦螢幕"},"webhookEventId":"01KE3J1Q63E8JT6QYXBZT6SB57","deliveryContext":{"isRedelivery":false},"timestamp":1767498636424,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"88238388b6ca4bc6a8ed135cb7928c4a","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [04/Jan/2026 03:50:40] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"text","id":"595033093204345429","quoteToken":"uZogsLSZKrDZzBcaJ-YyfxVhddSauQZ_7iRtQQ0eDWqXA5Ah2ylGBtQy9X7PbM32Ga5WNmylpsFKLwrujLPGbU9Q7Uukx4HJVUF2ln7MTQ7wE-wkYg0yTWxLrGGX-7vva_PSaFa8XdLgCMLzyfosHg","markAsReadToken":"f7BsUlIfGKngTXUVGd3fgVtHlj_eufLKHOtt891E1BNpz6P1o2-TylHvMqoKVm9BTsVKdZHGsJb-35iBgvb3wrljGAxh1hW9nz38zTGumakMt78yrgKs4JKWQO8WCzcgZNwdndOaDLAeWfl--AIeIoaFG1ljQVoFxd294pDH32oLCu9OySAC1PJ8J2gyZyZEtTiMdFBwINSZQzC3fUTU6g","text":"AI 我的CPU效能很好嗎"},"webhookEventId":"01KE3J2M752TV3BJ766EP8755S","deliveryContext":{"isRedelivery":false},"timestamp":1767498665704,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"fa361d670e0a44e39e58e71413406494","mode":"active"}]}


BODY:  {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"text","id":"595033093204345429","quoteToken":"uZogsLSZKrDZzBcaJ-YyfxVhddSauQZ_7iRtQQ0eDWqXA5Ah2ylGBtQy9X7PbM32Ga5WNmylpsFKLwrujLPGbU9Q7Uukx4HJVUF2ln7MTQ7wE-wkYg0yTWxLrGGX-7vva_PSaFa8XdLgCMLzyfosHg","markAsReadToken":"f7BsUlIfGKngTXUVGd3fgVtHlj_eufLKHOtt891E1BNpz6P1o2-TylHvMqoKVm9BTsVKdZHGsJb-35iBgvb3wrljGAxh1hW9nz38zTGumakMt78yrgKs4JKWQO8WCzcgZNwdndOaDLAeWfl--AIeIoaFG1ljQVoFxd294pDH32oLCu9OySAC1PJ8J2gyZyZEtTiMdFBwINSZQzC3fUTU6g","text":"AI 我的CPU效能很好嗎"},"webhookEventId":"01KE3J2M752TV3BJ766EP8755S","deliveryContext":{"isRedelivery":false},"timestamp":1767498665704,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"fa361d670e0a44e39e58e71413406494","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [04/Jan/2026 03:51:17] "POST / HTTP/1.1" 200 -
